# Discrete Fréchet Distance - Recursive or Linear?
This notebook presents two alternative implementations of the discrete Fréchet distance calculation. The first implementation is the recursive version as proposed by Eiter an Mannila. The second implementation is linear and emerged as an insight from the first.

We start by importing the required Python packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

from distances.discrete import euclidean
from typing import Callable
from numba import jit

Now we define the sample poly-lines for later use. These live in R<sup>2</sup> and we will use the euclidean distance to measure pointwise distances.

In [ ]:
p = np.array([[0.2, 2.0], 
              [1.5, 2.8], 
              [2.3, 1.6], 
              [2.9, 1.8], 
              [4.1, 3.1], 
              [5.6, 2.9], 
              [7.2, 1.3],
              [8.2, 1.1]])

In [ ]:
q = np.array([[0.3, 1.6], 
              [3.2, 3.0], 
              [3.8, 1.8],  
              [5.2, 3.1], 
              [6.5, 2.8], 
              [7.0, 0.8],
              [8.9, 0.6]])

## Recursive Implementation
Below is the recursive implementation of the discrete Fréchet distance. The `recursive_frechet` sets up a few global variables and then calls the `calculate` function. This function calls itself in five different locations. Following the return order of matrix indices, you can see that they are being generated in a sequential fashion: column first, then row. This is a great hint to improve on this algorithm with a sequential (non -recursive) one.

**Note**: the `@jit` annotations enable the Numba just-in-time compiler. When measuring performance of this code, always account for the initial compilation that makes the code run slightly slower the first time.

In [ ]:
@jit(nopython=True)
def calculate(ca: np.ndarray, i: int, j: int, dist_func: Callable[[np.ndarray, np.ndarray], float]) -> float:
    """
    Calculates the distance between p[i] and q[i]
    :param i: Index into poly-line p
    :param j: Index into poly-line q
    :return: Distance value
    """
    if ca[i, j] > -1.0:
        # Uncomment the line below to see when the code does the dynamic programming trick: reuses already-calculated values
        # print(i, j, "*")
        return ca[i, j]

    # Uncomment the line below to follow the order of recursive calls
    # print(i, j)
    d = dist_func(p[i], q[j])
    if i > 0 and j > 0:
        ca[i, j] = max(min(calculate(ca, i-1, j, dist_func),
                           calculate(ca, i-1, j-1, dist_func),
                           calculate(ca, i, j-1, dist_func)), d)
    elif i > 0 and j == 0:
        ca[i, j] = max(calculate(ca, i-1, 0, dist_func), d)
    elif i == 0 and j > 0:
        ca[i, j] = max(calculate(ca, 0, j-1, dist_func), d)
    elif i == 0 and j == 0:
        ca[i, j] = d
    else:
        ca[i, j] = np.infty
    # Uncomment the line below to follow the return order of the calculated values.
    # This is how the order of the returned coordinates was calculated in the Medium article.
    # print(i, j)
    return ca[i, j]


@jit(nopython=True)
def recursive_frechet(p: np.ndarray, q: np.ndarray, dist_func: Callable[[np.ndarray, np.ndarray], float]) -> float:
    """
    Calculates the Fréchet distance between poly-lines p and q
    This function implements the algorithm described by Eiter & Mannila
    :param p: Poly-line p
    :param q: Poly-line q
    :return: Distance value
    """
    n_p = p.shape[0]
    n_q = q.shape[0]
    ca = np.zeros((n_p, n_q), dtype=np.float64)
    ca.fill(-1.0)

    return calculate(ca, n_p - 1, n_q - 1, dist_func)

Use the cell below to execute or time the recursive call. Uncomment the first line to time the call, and remember to run it twice!

In [ ]:
#%%timeit
recursive_frechet(p, q, euclidean)

## Linear Implementation
The linear implementation uses two simple nested loops to do its job. This insight was drawn by studying how the recursive version works, and leads to a slightly better runtime and much improved call stack usage.

In [ ]:
@jit(nopython=True)
def linear_frechet(p: np.ndarray, q: np.ndarray, dist_func: Callable[[np.ndarray, np.ndarray], float]) -> float:
    n_p = p.shape[0]
    n_q = q.shape[0]
    ca = np.zeros((n_p, n_q), dtype=np.float64)

    for i in range(n_p):
        for j in range(n_q):
            d = dist_func(p[i], q[j])

            if i > 0 and j > 0:
                ca[i, j] = max(min(ca[i - 1, j],
                                   ca[i - 1, j - 1],
                                   ca[i, j - 1]), d)
            elif i > 0 and j == 0:
                ca[i, j] = max(ca[i - 1, 0], d)
            elif i == 0 and j > 0:
                ca[i, j] = max(ca[0, j - 1], d)
            elif i == 0 and j == 0:
                ca[i, j] = d
            else:
                ca[i, j] = np.infty
    return ca[n_p - 1, n_q - 1]

Same as before, for the linear implementation.

In [ ]:
%%timeit
linear_frechet(p, q, euclidean)

There is still room for improvement as we will see in another notebook. When calculating large polylines, most of the distance matrix is actually useless to calculate the implicit free-space diagram. Removing these distance calculations will further improve the calculation performance.